In [ ]:
def run_parallel_deep(data_shape, batched_train_data, batched_val_data, train_slices, 
             val_slices, BATCH_SIZE, EPOCHS, FUTURE_TARGET, MIN_DELTA, PATIENCE, KERNEL_SIZE, FILTERS, PADDING, LAYER_SIZE, DENSE_SIZE):
    model = compile_model_parallel_deep(data_shape, FUTURE_TARGET, KERNEL_SIZE, FILTERS, PADDING, LAYER_SIZE, DENSE_SIZE)
    training_history, training_time = fit_model_parallel_deep(model, batched_train_data, batched_val_data, train_slices, 
                                                val_slices, BATCH_SIZE, EPOCHS, MIN_DELTA, PATIENCE)
    return model, training_history, training_time

In [ ]:
def compile_model_parallel_deep(data_shape, FUTURE_TARGET, KERNEL_SIZE, FILTERS, PADDING, LAYER_SIZE, DENSE_SIZE):
    
    inputs = tf.keras.layers.Input(shape=data_shape, name='main_input')
    conv = tf.keras.layers.Conv1D(filters=20,kernel_size=KERNEL_SIZE,strides=1,padding=PADDING,activation='relu',dilation_rate=1)(inputs)
    conv = tf.keras.layers.Conv1D(filters=30,kernel_size=KERNEL_SIZE,strides=1,padding=PADDING,activation='relu',dilation_rate=2)(conv)
    conv = tf.keras.layers.Conv1D(filters=30,kernel_size=KERNEL_SIZE,strides=1,padding=PADDING,activation='relu',dilation_rate=4)(conv)
    conv = tf.keras.layers.Conv1D(filters=50,kernel_size=KERNEL_SIZE,strides=1,padding=PADDING,activation='relu',dilation_rate=8)(conv)
    lstm = tf.keras.layers.LSTM(LAYER_SIZE,return_sequences=True)(inputs) #true if deep lstm
    lstm = tf.keras.layers.Dense(DENSE_SIZE)(lstm)
    lstm = tf.keras.layers.LSTM(LAYER_SIZE,return_sequences=True)(lstm)
    lstm = tf.keras.layers.Dense(DENSE_SIZE)(lstm)
    lstm = tf.keras.layers.LSTM(LAYER_SIZE, return_sequences=False)(lstm)
    x = tf.keras.layers.Flatten()(conv)
    concat = tf.keras.layers.concatenate([lstm,x])
    x = tf.keras.layers.Dense(DENSE_SIZE, activation='relu')(concat)
    # x = tf.keras.layers.Dense(DENSE_SIZE, activation='relu')(x)
    main_output = tf.keras.layers.Dense(1,name='main_output')(x)
    model = tf.keras.models.Model(inputs=[inputs], outputs=[main_output])
    
    model.summary()
    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss={'main_output':'mae'},
                  loss_weights={'main_output':1}, metrics=['mae', 'mape', 'mse'])
    return model

In [ ]:
def fit_model_parallel_deep(model, batched_train_data, batched_val_data, train_slices, val_slices, BATCH_SIZE, EPOCHS, MIN_DELTA, PATIENCE):
    train_batches = train_slices // BATCH_SIZE 
    val_batches = val_slices // BATCH_SIZE
    # Splits the dataset into batches of this size: we perform gradiant descent once per batch
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=MIN_DELTA, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=True)

    start = timer()
    training_history = model.fit(batched_train_data, epochs=EPOCHS, 
                                 steps_per_epoch=train_batches,
                                 validation_data=batched_val_data,
                                 validation_steps=val_batches,
                                 callbacks=[es])
    end = timer()
    return training_history, end - start # time in seconds
